In [1]:
from tensorflow.python.tools import inspect_checkpoint as chkp
model_path = "/Users/claudecoulombe/git/tensorflow_basic_tutorial/model/"
chkp.print_tensors_in_checkpoint_file(model_path+'model.ckpt', tensor_name='', all_tensors=True)

tensor_name:  b1
[ 0.81852704  1.0076863   0.55985636 -1.259233    0.13556074 -1.2156585
 -0.12213396 -0.46883    -0.4676252   0.23242079 -0.30991337  0.10838123
 -1.4342843  -1.3888518   1.2242453  -0.58102405 -0.84635776  0.27092743
 -0.573409    0.2613535   1.1146485  -1.3061517   0.7094621  -0.04007692
 -0.8870731  -1.1728839  -1.6137373  -0.02936619 -0.06405356  0.64017075
 -1.1202021  -1.0206416  -1.5528885   1.3530838   1.4458653  -0.9197164
 -0.16445863  1.2468301  -1.3051164   0.42626104 -0.43164143 -0.03780001
  1.3321108   1.222957   -0.7517422   1.4872723   0.59574616 -0.29067308
  0.25031453  0.2157989  -0.38840505  0.8806445   1.7437965  -0.4650204
  0.30998355 -0.18925838 -1.8181741  -0.44355902 -0.50238425 -0.4727151 ]
tensor_name:  b1_1
[-0.09684636  0.09041497  0.35100055 -0.57734495  0.7018147   0.09669361
 -0.9120663   1.2373346  -1.8076465   0.96599984 -1.8510627  -0.5377109
 -0.7936769   0.7949273   0.44597512 -1.9096282   0.5324379  -0.97922426
  0.23666142 -0.32

In [9]:
# Testing model on fresh data, in fact synthesized data
# only for the purpose of getting an idea of the generalization of the model
import matplotlib.pyplot as plt 
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.python.framework import ops

# Create new data from the dataset "sonar.csv" modifying the features manually 
# creating 8 «synthetized» new data and save the file as new_sonar.csv
def read_dataset(file_name):
    dir_path = "/Users/claudecoulombe/git/tensorflow_basic_tutorial/"
    df = pd.read_csv(dir_path+file_name,delimiter=',')
    print("Nbr columns: ",len(df.columns))
    X = df[df.columns[0:60]].values
    y = df[df.columns[60]]
    # Encode the dependant variable
    encoder = LabelEncoder()
    encoder.fit(y)
    y = encoder.transform(y)
    Y = one_hot_encode(y)
    print("X.shape",X.shape)
    return (X,Y)
 
# Define the encoder function M => 1, R => 0
def one_hot_encode(labels):
    n_labels = len(labels)
#     n_unique_labels = len(np.unique(labels))
    n_unique_labels = 2
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels),labels] = 1
    return one_hot_encode

# Read the dataset
X, Y = read_dataset('new_sonar.csv')

n_dim = X.shape[1] 
n_class = 2

# Define the number of hidden layers an the
# number of neurons for each layer
n_hidden_1 = 60
n_hidden_2 = 60
n_hidden_3 = 60
n_hidden_4 = 60

def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activations
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with sigmoid activations
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    # Hidden layer with sigmoid activations
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.sigmoid(layer_3)
    # Hidden layer with RELU activations
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.relu(layer_4)
    # Output layer with linear activations
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out_b']
    return out_layer
 
tf.reset_default_graph() 

model_path = "/Users/claudecoulombe/git/tensorflow_basic_tutorial/model/"
new_saver = tf.train.import_meta_graph(model_path+'model.ckpt.meta')

sess = tf.Session()

with tf.Session() as sess:  
    new_saver.restore(sess, tf.train.latest_checkpoint(model_path))

    # Inputs and outputs
    x = tf.placeholder(tf.float32,[None, n_dim])
    y_ = tf.placeholder(tf.float32,[None, n_class])

    # define the weights and the biases for each layer
    weights = {
        'h1': sess.run("h1:0"),
        'h2': sess.run("h2:0"),
        'h3': sess.run("h3:0"),
        'h4': sess.run("h4:0"),
        'out': sess.run("out:0"),
        }
    biases = {
        'b1': sess.run("b1:0"),
        'b2': sess.run("b2:0"),
        'b3': sess.run("b3:0"),
        'b4': sess.run("b4:0"),
        'out_b': sess.run("out_b:0"),
        }

    # Call your model defined
    print("***************** Prediction on testing data *****************")
    print("**************************************************************")
    print("* 1 stands for M, (i.e. Mine) and 0 stands for R (i.e. Rock) *")
    print("**************************************************************")
    predicted_targets = sess.run(tf.argmax(multilayer_perceptron(x, weights, biases),1), feed_dict={x:X}).tolist()
    true_targets = [np.argmax(a_data_sample) for a_data_sample in Y]
    for true_target,predicted_target in zip(true_targets,predicted_targets):
        print("True target:",true_target, "Predicted target:",predicted_target,"Accurate prediction:",true_target==predicted_target)
    total_accurate = sum([1 for index in range(len(true_targets)) if true_targets[index] == predicted_targets[index]])
    print("Final accuracy:",str(round(total_accurate/len(true_targets),3)*100)+"%")

Nbr columns:  61
X.shape (8, 60)
INFO:tensorflow:Restoring parameters from /Users/claudecoulombe/git/tensorflow_basic_tutorial/model/model.ckpt
***************** Prediction on testing data *****************
**************************************************************
* 1 stands for M, (i.e. Mine) and 0 stands for R (i.e. Rock) *
**************************************************************
True target: 0 Predicted target: 0 Accurate prediction: True
True target: 0 Predicted target: 0 Accurate prediction: True
True target: 0 Predicted target: 0 Accurate prediction: True
True target: 1 Predicted target: 0 Accurate prediction: False
True target: 0 Predicted target: 0 Accurate prediction: True
True target: 0 Predicted target: 0 Accurate prediction: True
True target: 0 Predicted target: 1 Accurate prediction: False
True target: 0 Predicted target: 1 Accurate prediction: False
Final accuracy: 62.5%


With 62.5% , the model has shown some power of generalization.